# Fragestellung 1.3: Welche Gebiete in einer Gemeinde sind am stärksten betroffen?

Um diese Frage zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. <br>
[Link zum GWR](https://www.housing-stat.ch/de/index.html)<br>
[Link zur GWR Dokumentation](https://www.housing-stat.ch/de/docs/index.html)<br>
[Link zum GWR Download](https://www.housing-stat.ch/de/madd/public.html)<br>
<br>
Die Daten werden nach Gemeinde und Energiequelle gruppiert und dann die einzelnen Objekte gezählt. <br>
Anschliessend wird die resultierende Liste als Geopackage zur weiterverarbeitung bzw. Visualisierung exportiert.<br>
<br>
---
<i> CAS Spatial Data Analytics 2022 </i> ¦ <i> Kommunale Übersicht von Heizsystemen und Energieträgern in Wohngebäuden </i> ¦ <i> Stand: 22.09.2022  </i> ¦ <i> Entwickler: Jürg Reist </i>

### Notebook vorbereiten und die benötigten Daten aus dem GWR einlesen

In [35]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# Datenquelle definieren
url = urlopen("https://public.madd.bfs.admin.ch/ch.zip")

#lesen GWR Daten
zipfile = ZipFile(BytesIO(url.read()))
dfGWRSource = pd.read_csv(zipfile.open('gebaeude_batiment_edificio.csv'),
                         usecols=['GDEKT', 'GGDENAME', 'GKODE', 'GKODN', 'GENH1'],
                         sep='\t')

dfGWRSource.head()


,GDEKT,GGDENAME,GKODE,GKODN,GENH1
0,ZH,Aeugst am Albis,2679647.268,1237500.347,7530.0
1,ZH,Aeugst am Albis,2680635.900,1236936.229,7530.0
2,ZH,Aeugst am Albis,2679016.828,1235828.156,7530.0
3,ZH,Aeugst am Albis,2679677.869,1237453.256,7530.0
4,ZH,Aeugst am Albis,2679692.997,1237491.284,7530.0


In [44]:
#Column mit geeigneten Namen
dfGWRSource = dfGWRSource.rename(columns={"GDEKT": "Kanton",
                                          "GGDENAME": "Gemeinde",
                                          "GKODE": "lat",
                                          "GKODN": "lon",
                                          "GENH1":"Energiequelle"
                                          })
dfGWRSource.head()

,Kanton,Gemeinde,lat,lon,Energiequelle
0,ZH,Aeugst am Albis,2679647.268,1237500.347,7530.0
1,ZH,Aeugst am Albis,2680635.900,1236936.229,7530.0
2,ZH,Aeugst am Albis,2679016.828,1235828.156,7530.0
3,ZH,Aeugst am Albis,2679677.869,1237453.256,7530.0
4,ZH,Aeugst am Albis,2679692.997,1237491.284,7530.0


In [45]:
# Die Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
# Um die finale Visualisierung auf die drei definierten Energiequellen (Gas, Heizöl, Elektrizität) zu fokussieren, werden die Energiequellen zusammengefasst.
# Im GWR-Datenfile sind folgende Energiequellen definiert:
# 7501: 'Luft', 7510: 'Erdwärme (generisch', 7511: 'Erdwärmesonde', 7512: 'Erdregister', 7513: 'Wasser (Grundwasser, Oberflächenwasser, Abwasser)'
# 7520: 'Gas', 7530: 'Heizöl', 7540: 'Holz (generisch)', 7541: 'Holz (Stückholz)', 7542: 'Holz (Pellets)', 7543: 'Holz (Schnitzel)', 7550: 'Abwärme (innerhalb des Gebäudes)'
# 7560: 'Elektrizität', 7570: 'Sonne (thermisch)', 7580: 'Fernwärme (generisch)', 7581: 'Fernwärme (Hochtemperatur)', 7582: 'Fernwärme (Niedertemperatur)'
# 7598: 'Unbestimmt', 7599: 'Andere'

dfGWRSource['Energiequelle'] = dfGWRSource.Energiequelle.replace({
                                            7500: 'Keine',
                                            7501: 'Weitere',
                                            7510: 'Weitere',
                                            7511: 'Weitere',
                                            7512: 'Weitere',
                                            7513: 'Weitere',
                                            7520: 'Gas',
                                            7530: 'Heizöl',
                                            7540: 'Weitere',
                                            7541: 'Weitere',
                                            7542: 'Weitere',
                                            7543: 'Weitere',
                                            7550: 'Weitere',
                                            7560: 'Elektrizität',
                                            7570: 'Weitere',
                                            7580: 'Weitere',
                                            7581: 'Weitere',
                                            7582: 'Weitere',
                                            7598: 'Unbestimmt',
                                            7599: 'Weitere'                                           
                                            })
dfGWRSource.head()

,Kanton,Gemeinde,lat,lon,Energiequelle
0,ZH,Aeugst am Albis,2679647.268,1237500.347,Heizöl
1,ZH,Aeugst am Albis,2680635.900,1236936.229,Heizöl
2,ZH,Aeugst am Albis,2679016.828,1235828.156,Heizöl
3,ZH,Aeugst am Albis,2679677.869,1237453.256,Heizöl
4,ZH,Aeugst am Albis,2679692.997,1237491.284,Heizöl


In [46]:
dfGWRSource

,Kanton,Gemeinde,lat,lon,Energiequelle
0,ZH,Aeugst am Albis,2679647.268,1237500.347,Heizöl
1,ZH,Aeugst am Albis,2680635.900,1236936.229,Heizöl
2,ZH,Aeugst am Albis,2679016.828,1235828.156,Heizöl
3,ZH,Aeugst am Albis,2679677.869,1237453.256,Heizöl
4,ZH,Aeugst am Albis,2679692.997,1237491.284,Heizöl
...,...,...,...,...,...
3062669,JU,La Baroche,2582768.637,1251359.261,Heizöl
3062670,JU,La Baroche,2582743.072,1251406.135,Heizöl
3062671,JU,La Baroche,2584600.501,1251495.804,Gas
3062672,JU,La Baroche,2583393.948,1251448.443,Weitere


In [47]:
#nur Gas, Heizöl, Heizöl und Elektrizität als Energiequellen filtern
options = ['Gas', 'Heizöl', 'Elektrizität']
dfGWREnergiequelleGHE = dfGWRSource[(dfGWRSource['Energiequelle'].isin(options))] 

dfGWREnergiequelleGHE

                                                 

,Kanton,Gemeinde,lat,lon,Energiequelle
0,ZH,Aeugst am Albis,2679647.268,1237500.347,Heizöl
1,ZH,Aeugst am Albis,2680635.900,1236936.229,Heizöl
2,ZH,Aeugst am Albis,2679016.828,1235828.156,Heizöl
3,ZH,Aeugst am Albis,2679677.869,1237453.256,Heizöl
4,ZH,Aeugst am Albis,2679692.997,1237491.284,Heizöl
...,...,...,...,...,...
3062662,JU,La Baroche,2583088.040,1251351.424,Heizöl
3062668,JU,La Baroche,2583355.208,1251442.906,Heizöl
3062669,JU,La Baroche,2582768.637,1251359.261,Heizöl
3062670,JU,La Baroche,2582743.072,1251406.135,Heizöl


### Daten als Geopackage zur Weiterverarbeitung bzw. Visusliserung exportieren

In [48]:
gdf = gpd.GeoDataFrame(
    dfGWREnergiequelleGHE, geometry=gpd.points_from_xy(dfGWREnergiequelleGHE.lat, dfGWREnergiequelleGHE.lon)
)

In [49]:
gdf

,Kanton,Gemeinde,lat,lon,Energiequelle,geometry
0,ZH,Aeugst am Albis,2679647.268,1237500.347,Heizöl,POINT (2679647.268 1237500.347)
1,ZH,Aeugst am Albis,2680635.900,1236936.229,Heizöl,POINT (2680635.900 1236936.229)
2,ZH,Aeugst am Albis,2679016.828,1235828.156,Heizöl,POINT (2679016.828 1235828.156)
3,ZH,Aeugst am Albis,2679677.869,1237453.256,Heizöl,POINT (2679677.869 1237453.256)
4,ZH,Aeugst am Albis,2679692.997,1237491.284,Heizöl,POINT (2679692.997 1237491.284)
...,...,...,...,...,...,...
3062662,JU,La Baroche,2583088.040,1251351.424,Heizöl,POINT (2583088.040 1251351.424)
3062668,JU,La Baroche,2583355.208,1251442.906,Heizöl,POINT (2583355.208 1251442.906)
3062669,JU,La Baroche,2582768.637,1251359.261,Heizöl,POINT (2582768.637 1251359.261)
3062670,JU,La Baroche,2582743.072,1251406.135,Heizöl,POINT (2582743.072 1251406.135)


### Daten in ein GeoPackage-File exportieren

In [56]:
# nur ausgewählte Kantone exportiern, da sonst das Geopackage-File über 100MB wird. 
# GitHub hat eine Limitierung von 100MB  
options = ['BE', 'ZH', 'AG', 'VD', 'SG']
gdf_1 = gdf[(gdf['Kanton'].isin(options))] 

gdf_1

,Kanton,Gemeinde,lat,lon,Energiequelle,geometry
0,ZH,Aeugst am Albis,2679647.268,1237500.347,Heizöl,POINT (2679647.268 1237500.347)
1,ZH,Aeugst am Albis,2680635.900,1236936.229,Heizöl,POINT (2680635.900 1236936.229)
2,ZH,Aeugst am Albis,2679016.828,1235828.156,Heizöl,POINT (2679016.828 1235828.156)
3,ZH,Aeugst am Albis,2679677.869,1237453.256,Heizöl,POINT (2679677.869 1237453.256)
4,ZH,Aeugst am Albis,2679692.997,1237491.284,Heizöl,POINT (2679692.997 1237491.284)
...,...,...,...,...,...,...
2755937,VD,Yvonand,2547688.000,1183183.000,Elektrizität,POINT (2547688.000 1183183.000)
2755939,VD,Yvonand,2547566.000,1183228.000,Heizöl,POINT (2547566.000 1183228.000)
2755940,VD,Yvonand,2547735.000,1183173.000,Heizöl,POINT (2547735.000 1183173.000)
2755941,VD,Yvonand,2546692.000,1182258.000,Elektrizität,POINT (2546692.000 1182258.000)


In [58]:
# nur ausgewählte Kantone exportiern, da sonst das Geopackage-File über 100MB wird. 
# GitHub hat eine Limitierung von 100MB  
options = ['GR', 'SO', 'SZ', 'GE', 'JU', 'NE', 'SH', 'BS', 'ZG', 'AR', 'GL', 'UR', 'OW', 'NW', 'AI', 'BL', 'LU','TG',  'VS', 'FR', 'TI']
gdf_2 = gdf[(gdf['Kanton'].isin(options))] 

gdf_2

,Kanton,Gemeinde,lat,lon,Energiequelle,geometry
857601,LU,Doppleschwand,2646802.953,1207578.762,Elektrizität,POINT (2646802.953 1207578.762)
857602,LU,Doppleschwand,2646819.176,1207576.143,Elektrizität,POINT (2646819.176 1207576.143)
857606,LU,Doppleschwand,2646779.874,1207582.283,Elektrizität,POINT (2646779.874 1207582.283)
857607,LU,Doppleschwand,2646789.695,1207556.062,Elektrizität,POINT (2646789.695 1207556.062)
857609,LU,Doppleschwand,2646872.356,1207898.907,Elektrizität,POINT (2646872.356 1207898.907)
...,...,...,...,...,...,...
3062662,JU,La Baroche,2583088.040,1251351.424,Heizöl,POINT (2583088.040 1251351.424)
3062668,JU,La Baroche,2583355.208,1251442.906,Heizöl,POINT (2583355.208 1251442.906)
3062669,JU,La Baroche,2582768.637,1251359.261,Heizöl,POINT (2582768.637 1251359.261)
3062670,JU,La Baroche,2582743.072,1251406.135,Heizöl,POINT (2582743.072 1251406.135)


In [59]:
#Als GeoPackage-File exportieren - PART 1
gdf_1 = gdf_1.set_crs(2056, allow_override=True)
gdf_1.to_file('Daten/Gemeindeliste_1-3_part1.gpkg', driver='GPKG', limit=0)


In [60]:
#Als GeoPackage-File exportieren - PART 2
gdf_2 = gdf_2.set_crs(2056, allow_override=True)
gdf_2.to_file('Daten/Gemeindeliste_1-3_part2.gpkg', driver='GPKG', limit=0)